In [10]:
import pandas as pd
import os
import requests
from api import CENSUS_API_KEY
import ssl
from util.convertfile import *

In [11]:
pd.set_option("display.max_columns", None)

In [12]:
epa_df = pd.read_csv("~/Documents/UH/hon4350/water_quality/data/main/filtered_epa_tri.csv")
acs_df = pd.read_csv("~/Documents/UH/hon4350/water_quality/data/main/filtered_census_acs_5yr.csv")

In [15]:
acs_df["zip"] = acs_df["zip"].astype(str).str.zfill(5)
epa_df["zip"] = epa_df["zip"].astype(str).str.zfill(5)

merged_df = pd.merge(epa_df, acs_df, left_on=["zip", "year"], right_on=["zip", "year"], how="left")
merged_df

,year,trifd,frs_id,facility_name,street_address,city,county,state,zip,zip+4,latitude,longitude,horizontal_datum,parent_co_name,parent_co_db_num,federal_facility,industry_sector_code,industry_sector,chemical,elemental_metal_included,tri_chemical/compound_id,classification,metal,metal_category,carcinogen,unit_of_measure,released_water,total_population,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,pct_white,pct_black,pct_asian,pct_hispanic
0,2016,7545WMLSSP828NH,1.100128e+11,MELISSA PLANT,8280 N HWY 5,MELISSA,COLLIN,TX,75454,0,33.310199,-96.559305,NAD83,NaN,NaN,NO,327,Nonmetallic Mineral Product,Lead,NO,0007439921,PBT,YES,Elemental metals,YES,Pounds,0.001,3171.0,72147.000,0.336,2347.0,100.0,27.0,685.0,0.740,0.032,0.009,0.216
1,2016,77028NLCXX46HME,1.100253e+11,NELCO,4600 HOMESTEAD RD,HOUSTON,HARRIS,TX,77028,0,29.803290,-95.301800,NAD83,NELCO,NaN,NO,332,Fabricated Metals,Lead,NO,0007439921,PBT,YES,Elemental metals,YES,Pounds,10.000,14530.0,29098.000,0.456,246.0,10248.0,0.0,3758.0,0.017,0.705,0.000,0.259
2,2016,7704WJRSYV7641W,1.100603e+11,JERSEY VILLAGE PLANT,7641 WRIGHT RD,HOUSTON,HARRIS,TX,77041,0,29.889700,-95.589310,NAD83,ARGOS USA CORP,969962146.0,NO,327,Nonmetallic Mineral Product,Lead,NO,0007439921,PBT,YES,Elemental metals,YES,Pounds,0.003,24011.0,60625.750,0.387,5419.0,2555.0,2899.0,12738.0,0.226,0.106,0.121,0.531
3,2016,7840WBCKYT151SU,1.100676e+11,BUCKEYE TEXAS PROCESSING LLC-CORPUS CHRISTI,7209 UP RIVER RD,CORPUS CHRISTI,NUECES,TX,78409,0,27.816200,-97.502610,NAD83,BUCKEYE PARTNERS LP,112569892.0,NO,324,Petroleum,Lead,NO,0007439921,PBT,YES,Elemental metals,YES,Pounds,3.280,2453.0,51971.000,0.392,485.0,44.0,0.0,1924.0,0.198,0.018,0.000,0.784
4,2016,77536DSPSL2525B,1.100177e+11,TM DEER PARK SERVICES LP,2525 BATTLEGROUND RD,DEER PARK,HARRIS,TX,77536,0,29.734270,-95.089860,NAD83,TEXAS MOLECULAR LP,42400759.0,NO,325,Chemicals,Lead,NO,0007439921,PBT,YES,Elemental metals,YES,Pounds,0.100,24498.0,68466.750,0.396,15718.0,213.0,276.0,7924.0,0.642,0.009,0.011,0.323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,2023,77705NTRNT500IN,1.100641e+11,AZZ GALVANIZING SERVICES-BEAUMONT,5898 INDUSTRIAL RD,BEAUMONT,JEFFERSON,TX,77705,0,29.994196,-94.197701,NAD83,AZZ INC,8012148.0,NO,332,Fabricated Metals,Zinc compounds,NO,N982,TRI,YES,Metal complound categories,NO,Pounds,87.090,25735.0,61476.250,0.404,8230.0,12369.0,544.0,4089.0,0.320,0.481,0.021,0.159
3246,2023,78263WSMMR9599G,1.100005e+11,CALAVERAS POWER STATION,12940 S US HIGHWAY 181,SAN ANTONIO,BEXAR,TX,78223,0,29.308764,-98.321969,NAD83,NaN,NaN,NO,2211,Electric Utilities,Zinc compounds,NO,N982,TRI,YES,Metal complound categories,NO,Pounds,7020.000,53770.0,51842.000,0.425,7209.0,2682.0,579.0,42874.0,0.134,0.050,0.011,0.797
3247,2023,75702RXHDN705SO,1.100005e+11,REX-HIDE INDUSTRIES INC,705 S LYONS ST,TYLER,SMITH,TX,75701,0,32.340950,-95.326210,NAD83,REX-HIDE INDUSTRIES INC,4320180.0,NO,326,Plastics and Rubber,Zinc compounds,NO,N982,TRI,YES,Metal complound categories,NO,Pounds,55.000,30796.0,60040.000,0.456,14543.0,7270.0,373.0,7050.0,0.472,0.236,0.012,0.229
3248,2023,77229ZTCMN12433,1.100005e+11,AZZ GALVANIZING SERVICES-HOUSTON,7407 C E KING PKWY,HOUSTON,HARRIS,TX,77044,0,29.832649,-95.211053,NAD83,AZZ INC,8012148.0,NO,332,Fabricated Metals,Zinc compounds,NO,N982,TRI,YES,Metal complound categories,NO,Pounds,989.360,55746.0,92487.375,0.412,10436.0,17963.0,803.0,24747.0,0.187,0.322,0.014,0.444


In [25]:
missing_zips = merged_df[merged_df["gini_index"].isna()]["zip"].unique()
print(missing_zips)

['75668' '78419' '77385' '78728' '78359' '77507' '76657' '75692' '76052'
 '75261' '76247' '76448' '76084' '78736' '75934' '77978' '77655' '78602'
 '75691' '78726' '77423' '76539' '75109' '75147' '75859' '75846' '77615'
 '75207' '77463' '75706' '75831' '75440' '78725' '75569' '77396' '78503'
 '77484' '77960' '77481']


In [18]:
merged_df.describe()

,year,frs_id,zip+4,latitude,longitude,parent_co_db_num,industry_sector_code,released_water,total_population,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,pct_white,pct_black,pct_asian,pct_hispanic
count,3250.000000,3.250000e+03,3250.000000,3250.000000,3250.000000,2.453000e+03,3250.000000,3250.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000
mean,2019.269231,1.100222e+11,24.497231,30.952460,-96.238548,2.389600e+08,459.866154,271.222741,22828.131256,62512.182683,0.414044,8930.803447,3179.877693,810.599271,9335.307922,0.424931,0.144197,0.025202,0.381913
std,2.340868,2.497551e+07,267.825575,1.742591,1.626134,3.408362e+08,524.356129,1987.373114,16092.275118,24071.562329,0.042957,7472.342724,3824.866149,2080.086688,10103.545399,0.240281,0.144423,0.041029,0.232845
min,2016.000000,1.100005e+11,0.000000,25.915560,-106.585052,1.024314e+06,311.000000,0.001000,708.000000,21504.750000,0.306000,7.000000,0.000000,0.000000,8.000000,0.002000,0.000000,0.000000,0.003000
25%,2017.000000,1.100005e+11,0.000000,29.731000,-97.178060,7.317480e+06,326.000000,0.215500,10960.000000,45313.000000,0.383000,3138.000000,555.000000,49.000000,2510.000000,0.222000,0.041000,0.004000,0.182000
50%,2019.000000,1.100127e+11,0.000000,30.233529,-95.887330,7.874948e+07,331.000000,2.288500,20130.000000,56960.400000,0.408000,7168.000000,1830.000000,212.000000,6165.000000,0.447000,0.101000,0.010000,0.340000
75%,2021.000000,1.100375e+11,0.000000,32.662490,-95.126944,1.941232e+08,332.000000,21.725000,31827.000000,75961.800000,0.440000,13187.000000,4326.000000,646.000000,12664.000000,0.606000,0.189000,0.028000,0.544000
max,2023.000000,1.100717e+11,3257.000000,35.951944,-93.734518,9.699621e+08,4247.000000,54755.560000,131860.000000,243401.833000,0.592000,53707.000000,34863.000000,31688.000000,83748.000000,0.931000,0.864000,0.344000,0.968000


In [21]:
merged_df['gini_index'].isnull().any()

True

In [17]:
merged_df.to_csv("~/Documents/UH/hon4350/water_quality/data/main/main.csv")
merged_df.to_excel("~/Documents/UH/hon4350/water_quality/data/main/main.xlsx")